In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime
import json

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [2]:
from FormatEvents_Functions import *

In [3]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [4]:
durations = ['0.5', '1', '2', '3', '6', '12', '24']

### UKCP18 data

### Join together lists for different ensemble members

In [5]:
events_props_dict_present = []
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
for em in ems_present:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_props_dict_{em}_NEW.pickle", 'rb') as handle:
        one_events_props_dict_present = pickle.load(handle)    
        events_props_dict_present = events_props_dict_present + one_events_props_dict_present
    
## Join into one dataframe    
present = pd.DataFrame(events_props_dict_present)
present['Climate'] = 'Present'    

In [6]:
events_props_dict_future = []
ems_future = ['bb189', 'bb195', 'bb192', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb219', 'bb211', 'bb216'] #  ,
for em in ems_future:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_{em}_NEW.pickle", 'rb') as handle:
        one_events_props_dict_future = pickle.load(handle)    
    events_props_dict_future = events_props_dict_future + one_events_props_dict_future

    ## Join into one dataframe
future = pd.DataFrame(events_props_dict_future)
future['Climate'] = 'Future'

In [7]:
df_long = pd.concat([present, future])

# # Add D variable (day of year) and date
df_long['D'] = (df_long['theta'] * 365.25) / (2 * np.pi)
df_long['date'] = df_long.apply(lambda row: date_from_D(row['D'], row['year']), axis=1)
df_long['season'] = df_long['date'].apply(get_season)

bin_edges = [0, 7.0, 16, 166.5]  # Example bin edges
duration_labels = ['<=7hr', '7-16hr', '16hr+']  # Corresponding labels for the bins

# # Apply the function to the entire column
df_long['duration_category_onemore'] = apply_dur_category_to_column(df_long, 'duration', bin_edges, duration_labels)

df_long.to_csv(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics.csv", index=False)

### NIMROD data

In [7]:
with open(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/event_props_dict_NEW.pickle", 'rb') as handle:
    events_props_dict_nimrod = pickle.load(handle)

In [8]:
nimrod = pd.DataFrame(events_props_dict_nimrod)

In [9]:
# Add D variable (day of year) and date
nimrod['D'] = (nimrod['theta'] * 365.25) / (2 * np.pi)
nimrod['date'] = nimrod.apply(lambda row: date_from_D(row['D'], row['year']), axis=1)
nimrod['season'] = nimrod['date'].apply(get_season)

bin_edges = [0, 7.0, 16, 166.5]  # Example bin edges
duration_labels = ['<=7hr', '7-16hr', '16hr+']  # Corresponding labels for the bins

# # Apply the function to the entire column
nimrod['duration_category_onemore'] = apply_dur_category_to_column(nimrod, 'duration', bin_edges, duration_labels)

nimrod.to_csv(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/all_events_characteristics.csv", index=False)

In [11]:
# # Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
# nimrod['dur_for_which_this_is_amax'] = nimrod['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# # Flatten the lists and count occurrences of each number
# all_numbers = [num for sublist in nimrod['dur_for_which_this_is_amax'] for num in sublist]
# number_counts = pd.Series(all_numbers).value_counts()

# # Show the result
# print(number_counts)

24     19462
0.5    19363
12     14560
6      11379
2       8136
1       7536
3       6830
dtype: int64
